In [1]:
import numpy as np
import pandas as pd
from clean import manifest_clinical_merge
from collections import Counter

In [2]:
pd.set_option('display.max_columns', 100)

# Data Prep

In [3]:
manifest_df = pd.read_csv('../Manifest_Data/GCD_TARGET_Data_Manifest_AML_NBL_WT_RT.csv')

wt_disc_df = pd.read_excel('../Clinical_Data/TARGET_WT_ClinicalData_Discovery_20160714_public.xlsx')
aml_disc_df = pd.read_excel('../Clinical_Data/TARGET_AML_ClinicalData_20160714.xlsx')
nbl_disc_df = pd.read_excel('../Clinical_Data/TARGET_NBL_ClinicalData_20151124.xlsx')

WT_df = manifest_clinical_merge(manifest_df, wt_disc_df, 'TARGET-WT')
AML_df = manifest_clinical_merge(manifest_df, aml_disc_df, 'TARGET-AML')
NBL_df = manifest_clinical_merge(manifest_df, nbl_disc_df, 'TARGET-NBL')

/home/ubuntu/projects/RNAseq_Cancer_Biomarkers/scripts/clean.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  target_df['TARGET USI'] = target_df.loc[:, 'entity_submitter_id'].apply(lambda x: x[:16])


In [4]:
from clean import assay_transpose

In [5]:
assay_df = pd.read_csv('../Expn_Data/TARGET_NBL_AML_RT_WT_TMMCPM_log2_Norm_Counts.csv.zip')
assay_t_df = assay_transpose(assay_df)

In [6]:
from clean import assay_clinical_merge

In [7]:
AML_genes = assay_clinical_merge(assay_t_df, AML_df)
WT_genes = assay_clinical_merge(assay_t_df, WT_df)
NBL_genes = assay_clinical_merge(assay_t_df, NBL_df)

In [8]:
from model_comp import data_prep_columns

In [9]:
# import xgboost

In [10]:
df, __ = data_prep_columns(AML_genes, 'neither')

In [11]:
print(df.info())
print('\n')
AML_genes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137 entries, 3 to 185
Columns: 21492 entries, TARGET USI to Low Risk
dtypes: float64(3), int64(7), object(21482)
memory usage: 22.5+ MB
None


<class 'pandas.core.frame.DataFrame'>
Int64Index: 187 entries, 0 to 186
Columns: 21491 entries, TARGET USI to __alignment_not_unique
dtypes: float64(3), int64(6), object(21482)
memory usage: 30.7+ MB


## train/test split selection

- testing - 44 samples from composite_code
- training - 93 samples from composite_code
- N = 137

In [12]:
testing = pd.read_csv('../composite_code/rnotebook/data/TARGET_AML_Testing_Samples.csv')
training = pd.read_csv('../composite_code/rnotebook/data/TARGET_AML_Training_Samples.csv')

In [13]:
df.head()

,TARGET USI,Diagnostic ID,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,Year of Diagnosis,Year of Last Follow Up,Protocol,WBC at Diagnosis,Bone marrow leukemic blast percentage (%),Peripheral blasts (%),CNS disease,Chloroma,FAB Category,t(6;9),t(8;21),t(3;5)(q25;q34),t(6;11)(q27;q23),t(9;11)(p22;q23),t(10;11)(p11.2;q23),t(11:19)(q23:p13.1),inv(16),del5q,del7q,del9q,monosomy 5,monosomy 7,trisomy 8,trisomy 21,MLL,Minus Y,Minus X,Cytogenetic Code Other,Cytogenetic Complexity,Primary Cytogenetic Code,ISCN,FLT3/ITD positive?,FLT3/ITD allelic ratio,FLT3 PM,NPM mutation,CEBPA mutation,WT1 mutation,c-Kit Mutation Exon 8,c-Kit Mutation Exon 17,MRD at end of course 1,...,ENSG00000280913.1,ENSG00000280916.1,ENSG00000280927.1,ENSG00000280936.1,ENSG00000280953.1,ENSG00000280987.1,ENSG00000281005.1,ENSG00000281021.1,ENSG00000281026.1,ENSG00000281100.1,ENSG00000281103.1,ENSG00000281106.1,ENSG00000281128.1,ENSG00000281131.1,ENSG00000281183.1,ENSG00000281189.1,ENSG00000281195.1,ENSG00000281205.1,ENSG00000281207.1,ENSG00000281332.1,ENSG00000281344.1,ENSG00000281357.1,ENSG00000281358.1,ENSG00000281376.1,ENSG00000281383.1,ENSG00000281392.1,ENSG00000281398.1,ENSG00000281404.1,ENSG00000281406.1,ENSG00000281433.1,ENSG00000281445.1,ENSG00000281468.1,ENSG00000281490.1,ENSG00000281501.1,ENSG00000281527.1,ENSG00000281538.1,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1,__no_feature,__ambiguous,__alignment_not_unique,Low Risk
3,TARGET-20-PADZCG,09A,Female,Unknown,Not Hispanic or Latino,5325,Relapse,314,Dead,536,1996,1998,CCG-2961,29.3,58,5,No,No,M4,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,0,Normal,"46,XX[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-2.42284,0.314777,1.74517,-6.92714,-6.12136,1.40683,0.995511,-0.223406,2.2914,-0.613341,3.9011,1.61224,-4.6821,-6.92714,0.8846,-2.4712,0.732776,0.678398,0.391151,1.82242,1.4786,1.37983,0.894175,-1.11056,-0.330849,0.0915962,1.95643,-0.547052,-6.92714,-1.45498,1.12314,-2.93057,2.9388,1.76878,0.580973,-0.832714,-1.78095,0.132623,-0.682824,5.46561,-0.330849,-1.84323,-0.801464,-0.364707,2.1646,-0.152689,17.1243,14.6061,17.8527,0
5,TARGET-20-PAEAKL,09A,Female,Black or African American,Not Hispanic or Latino,5218,Relapse,134,Dead,163,1997,1997,CCG-2961,96.1,80,81,Yes,No,M5,No,No,No,No,Yes,No,Yes,No,No,No,No,No,No,No,No,Yes,No,No,,1,MLL,"46,XX,t(11:19)(q23:p13.1)[20]",No,NaN,No,No,No,No,Not done,Not done,Unknown,...,-5.80459,-6.92714,1.03526,-6.92714,-6.92714,0.113774,1.32433,-4.41384,0.664587,-6.92714,1.21435,-6.92714,-6.92714,-5.80459,-0.0939682,-2.53384,-1.48807,-4.14335,-1.05214,0.929309,0.340124,0.95113,1.78753,-0.202203,6.32096,-0.544956,1.04881,0.496696,-6.92714,-2.79931,-0.914236,-1.94482,1.12118,3.16766,-2.61702,-1.89207,-3.1249,1.05554,-1.94482,3.83862,-2.17756,-0.717812,-0.88817,1.75482,0.456577,0.965496,16.7942,15.3635,19.6702,0
9,TARGET-20-PAEERJ,09A,Female,White,Not Hispanic or Latino,5082,Relapse,259,Dead,373,1997,1998,CCG-2961,71.9,60,41,No,No,M4,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,,1,Other,"46,XX,t(3:5)(q23:q33)[19]/46,XX[1]",Yes,0.55,No,No,No,Yes,Not done,Not done,Unknown,...,-3.82289,-3.00939,2.17987,-6.92714,-6.92714,1.94937,-1.00327,0.722174,3.02575,-3.82289,4.43513,-3.49809,-6.92714,-6.92714,1.32028,-4.50961,1.67087,-2.35434,-0.680935,1.47645,1.32543,0.512625,0.884345,-0.488087,-1.00327,-1.25643,2.91237,-0.209853,-6.92714,-1.35175,-0.0681887,-2.81572,-0.0817453,0.557041,0.30953,-2.56667,-1.85807,-0.470147,1.27314,6.07709,0.768241,-0.600677,-1.56366,-0.506252,2.3282,-2.90931,16.916,14.5652,17.571,0
10,TARGET-20-PAEFGR,09A,Female,White,Not Hispanic or Latino,549,Relapse,497,Alive,4022,1997,2008,CCG-2961,51.2,75,41,No,No,M2,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,,1,inv(16),"46,XX,inv(16)(p13q22)[21]/46,XX[1]",No,NaN,No,No,No

In [14]:
training_genes = list(training['x'])
testing_genes = list(testing['x'])

In [15]:
train_df = df[df['TARGET USI'].isin(training_genes)]
test_df = df[df['TARGET USI'].isin(testing_genes)]

In [16]:
print(train_df.info())
print('\n')
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 3 to 185
Columns: 21492 entries, TARGET USI to Low Risk
dtypes: float64(3), int64(7), object(21482)
memory usage: 15.3+ MB
None


<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 10 to 183
Columns: 21492 entries, TARGET USI to Low Risk
dtypes: float64(3), int64(7), object(21482)
memory usage: 7.2+ MB
None


### DF preparation
data has been split, now prepare for modeling

modeling on 21,404 genes

In [17]:
model_columns = df.iloc[:, 84:-4].columns

In [18]:
model_columns

Index(['ENSG00000000003.13', 'ENSG00000000005.5', 'ENSG00000000419.11',
       'ENSG00000000457.12', 'ENSG00000000460.15', 'ENSG00000000938.11',
       'ENSG00000000971.14', 'ENSG00000001036.12', 'ENSG00000001084.9',
       'ENSG00000001167.13',
       ...
       'ENSG00000281555.1', 'ENSG00000281571.1', 'ENSG00000281628.1',
       'ENSG00000281649.1', 'ENSG00000281691.1', 'ENSG00000281706.1',
       'ENSG00000281741.1', 'ENSG00000281789.1', 'ENSG00000281896.1',
       'ENSG00000281912.1'],
      dtype='object', length=21404)

In [20]:
X_train = train_df[model_columns].astype(float)
y_train = train_df.iloc[:, -1] #will scramble these

X_test = test_df[model_columns].astype(float)
y_test = test_df.iloc[:, -1]

X = df[model_columns]
y = df.iloc[:, -1]

In [21]:
len(y_train)

93

In [22]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from xgboost import XGBClassifier
import random
from sklearn.model_selection import permutation_test_score
import datetime

In [23]:
xgb = XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 700, random_state=8, n_jobs=-1)
rf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=8, n_jobs=-1)

In [24]:
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

p_rf = rf.predict_proba(X_test)
p_xgb = xgb.predict_proba(X_test)
p_rf_acc = rf.predict(X_test)
p_xgb_acc = xgb.predict(X_test)

rf_ll = log_loss(y_test, p_rf)
xgb_ll = log_loss(y_test, p_xgb)
rf_acc = accuracy_score(y_test, p_rf_acc)
xgb_acc = accuracy_score(y_test, p_xgb_acc)

print(f'RF Performance: Accuracy: {rf_acc} Log Loss: {rf_ll}')
print('\n')
print(f'XGB Performance: Accuracy: {xgb_acc} Log Loss: {xgb_ll}')

RF Performance: Accuracy: 0.8863636363636364 Log Loss: 0.37249402770254797


XGB Performance: Accuracy: 0.9090909090909091 Log Loss: 0.21215451328845863


These are my benchmarks

RF Performance: Accuracy: 0.8863636363636364 Log Loss: 0.37249402770254797

XGB Performance: Accuracy: 0.9090909090909091 Log Loss: 0.21215451328845863

### Permutation Tests

In [31]:
print(datetime.datetime.utcnow())
print('\n')

true_score, perm_scores, p_value =  permutation_test_score(rf, X, y, 
                                   scoring= 'neg_log_loss', cv=5, 
                                   n_permutations=50, n_jobs=-1, verbose=1)

print('\n')
print(datetime.datetime.utcnow())

2019-03-30 15:48:05.676510




[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.7min




2019-03-30 15:51:41.194756


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  3.5min finished


In [36]:
5000 / 50 * 4 / 60 
# 5-7 hours to run

6.666666666666667

### Lasso Regression

then train on those features --> then do permutation test

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold

In [46]:
final_val = df.sample(frac=0.1)

final_X = final_val[model_columns]
final_y = final_val.iloc[:, -1]

data = df.drop(index= final_val.index)
X = data[model_columns]
y = data.iloc[:, -1]

In [51]:
len(X) == len(y)

True

In [52]:
# log_model = LogisticRegression(penalty='l1', solver='saga')
# log_model.fit(X_train, y_train)
# y_pred = log_model.predict_proba(X_test)
# log_loss(y_test, y_pred)

In [53]:
# log_model.coef_

In [56]:
log_model = LogisticRegression(penalty='l1', solver='saga', max_iter=10000)
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
model_weights = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    log_model.fit(X_train, y_train)
    y_pred = log_model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    
    ll_performance.append(log_ll)
    model_weights.append(log_model.coef_)    

Did a 5-split kfold validation to find optimal weights for the logistic regression

In [57]:
ll_performance

[0.1244686704975814,
 0.16839041415802136,
 0.28790563194982294,
 0.2974177382044773,
 0.09307145514906583]

In [100]:
# len(model_weights)
# model_weights[0]

Here I am taking the average weight of all the coefs of the kfolds + also find the weight of the coefs
for all kfolds to find genes that are deemed somewhat important.

In [80]:
average_weight = np.mean(model_weights, axis=0)[0]

In [87]:
def important_gene_mask(columns, coefs):
    mask = coefs != 0
    important_genes = columns[mask[0]]
    print(len(important_genes))
    return important_genes

In [101]:
lasso_k1 = set(important_gene_mask(model_columns, model_weights[0]))
lasso_k2 = set(important_gene_mask(model_columns, model_weights[1]))
lasso_k3 = set(important_gene_mask(model_columns, model_weights[2]))
lasso_k4 = set(important_gene_mask(model_columns, model_weights[3]))
lasso_k5 = set(important_gene_mask(model_columns, model_weights[4]))

445
491
358
488
424


In [104]:
total_gene_union = set.union(lasso_k1, lasso_k2, lasso_k3, lasso_k4, lasso_k5)
len(total_gene_union)

976

976 possible genes the lasso regression thinks is important depending on the kfold it got..

In [105]:
total_gene_intersection = set.intersection(lasso_k1, lasso_k2, lasso_k3, lasso_k4, lasso_k5)
len(total_gene_intersection)

114

114 genes that all five lasso models agree on

### Now lets use these 976 genes to model RF or XGB

In [108]:
lasso_columns = list(total_gene_union)

In [109]:
X_2 = data[lasso_columns]
y_2 = data.iloc[:, -1]

In [114]:
X_2 = X_2.apply(pd.to_numeric)

In [115]:
xgb = XGBClassifier(learning_rate = 0.01, max_depth = 3, n_estimators = 700, random_state=8, n_jobs=-1)
rf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=8, n_jobs=-1)
kf = KFold(n_splits=5, shuffle=True)

rf_ll_performance = []
rf_acc_performance = []

xgb_ll_performance = []
xgb_acc_performance = []


for train_index, test_index in kf.split(X_2):
    X_train, X_test = X_2.iloc[train_index], X_2.iloc[test_index]
    y_train, y_test = y_2.iloc[train_index], y_2.iloc[test_index]
    
    rf.fit(X_train, y_train)
    xgb.fit(X_train, y_train)

    p_rf = rf.predict_proba(X_test)
    p_xgb = xgb.predict_proba(X_test)
    p_rf_acc = rf.predict(X_test)
    p_xgb_acc = xgb.predict(X_test)

    rf_ll = log_loss(y_test, p_rf)
    xgb_ll = log_loss(y_test, p_xgb)
    rf_acc = accuracy_score(y_test, p_rf_acc)
    xgb_acc = accuracy_score(y_test, p_xgb_acc)
    
    rf_ll_performance.append(rf_ll)
    rf_acc_performance.append(rf_acc)
    xgb_ll_performance.append(xgb_ll)
    xgb_acc_performance.append(xgb_acc)

In [117]:
from IPython.display import display

In [120]:
display(rf_ll_performance)

display(xgb_ll_performance)

display(rf_acc_performance)

display(xgb_acc_performance)

[0.20375300910564062,
 0.22029852207454778,
 0.30473843595013117,
 0.2513845462646569,
 0.18471674226032508]

[0.26560529809445144,
 0.16962465520948172,
 0.3230702378228307,
 0.19044157241781554,
 0.08915616905627151]

[0.96, 0.96, 0.88, 0.9583333333333334, 1.0]

[0.84, 0.96, 0.84, 0.9166666666666666, 1.0]

Really good accuracy and model performance here...

- Accuracy of 100% !!!

In [128]:
rf_kfold_5_df = pd.DataFrame({'rf prediction_0': p_rf[:, 0], 'rf prediction_1': p_rf[:, 1],
              'actual': y_test})
xgb_kfold_5_df = pd.DataFrame({'xgb prediction_0': p_xgb[:, 0], 'xgb prediction_1': p_xgb[:, 1],
              'actual': y_test})

In [129]:
rf_kfold_5_df.head()

,rf prediction_0,rf prediction_1,actual
10,0.133,0.867,1
24,0.894,0.106,0
30,0.916,0.084,0
32,0.154,0.846,1
35,0.878,0.122,0


In [130]:
xgb_kfold_5_df.head()

,xgb prediction_0,xgb prediction_1,actual
10,0.037930,0.962070,1
24,0.987535,0.012465,0
30,0.987362,0.012638,0
32,0.114972,0.885028,1
35,0.661000,0.339000,0


In [136]:
np.mean(rf_ll_performance)

0.2329782511310603

### Next steps

- tune model with kfolds
- fit on all the "training" data
- validate with the final "validation" set
- write a note on the 114 features deemed important by each round of lasso
- find "feature importance" of XGB, RF forest 
- permutation test with the 976 features